# Identifying the Number of Hackers 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

## Load Data

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName('cluster').getOrCreate()

In [4]:
dataset = spark.read.csv('datasets/hack_data.csv', inferSchema=True, header=True)

In [5]:
dataset.head(1)

[Row(Session_Connection_Time=8.0, Bytes Transferred=391.09, Kali_Trace_Used=1, Servers_Corrupted=2.96, Pages_Corrupted=7.0, Location='Slovenia', WPM_Typing_Speed=72.37)]

In [6]:
dataset.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



Every field is in a numerical column except `Location`. But, the forensic engineer told us that the hacker most likely used VPN, so the value in the column is not valid anymore. We can drop that later.

## Feature Selection and Scaling

In [7]:
from pyspark.ml.feature import VectorAssembler

In [9]:
assembler = VectorAssembler( inputCols=['Session_Connection_Time', 'Bytes Transferred', 
                                        'Kali_Trace_Used', 'Servers_Corrupted',
                                        'Pages_Corrupted', 'WPM_Typing_Speed'], outputCol = 'features')

In [10]:
final_data = assembler.transform( dataset)

In [11]:
final_data.printSchema()

root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)
 |-- features: vector (nullable = true)



In [12]:
from pyspark.ml.feature import StandardScaler

In [14]:
scaler = StandardScaler( withMean=True, inputCol='features', outputCol='scaledFeatures')

In [16]:
scaler_model = scaler.fit( final_data)

In [17]:
scaled_data = scaler_model.transform( final_data)

In [21]:
scaled_data.select('features', 'scaledFeatures').show()

+--------------------+--------------------+
|            features|      scaledFeatures|
+--------------------+--------------------+
|[8.0,391.09,1.0,2...|[-1.5622280401844...|
|[20.0,720.99,0.0,...|[-0.7104514131868...|
|[31.0,356.32,1.0,...|[0.07034382822759...|
|[2.0,228.08,1.0,2...|[-1.9881163536832...|
|[20.0,408.5,0.0,3...|[-0.7104514131868...|
|[1.0,390.69,1.0,2...|[-2.0590977392663...|
|[18.0,342.97,1.0,...|[-0.8524141843531...|
|[22.0,101.61,1.0,...|[-0.5684886420205...|
|[15.0,275.53,1.0,...|[-1.0653583411025...|
|[12.0,424.83,1.0,...|[-1.2783024978519...|
|[15.0,249.09,1.0,...|[-1.0653583411025...|
|[32.0,242.48,0.0,...|[0.14132521381072...|
|[23.0,514.54,0.0,...|[-0.4975072564374...|
|[9.0,284.77,0.0,3...|[-1.4912466546013...|
|[27.0,779.25,1.0,...|[-0.2135817141049...|
|[12.0,307.31,1.0,...|[-1.2783024978519...|
|[21.0,355.94,1.0,...|[-0.6394700276037...|
|[10.0,372.65,0.0,...|[-1.4202652690181...|
|[20.0,347.23,1.0,...|[-0.7104514131868...|
|[22.0,456.57,0.0,...|[-0.568488

## Train the Model

Now, we will create KMeans models with two different k values, k = 2 and k =3. The reason for this is because we want to figure out was it two hackers or three hackers. 

In [22]:
from pyspark.ml.clustering import KMeans

In [23]:
kmeans2 = KMeans( featuresCol='scaledFeatures', k = 2)
kmeans3 = KMeans( featuresCol='scaledFeatures', k = 3)

In [24]:
model_k2 = kmeans2.fit( scaled_data)
model_k3 = kmeans3.fit( scaled_data)

## Evaluate the Model

Now comes for the **trickiest part**: Trying to figure out if it was two hackers or three hackers.

But, we are given some useful hint. Remember that **the hackers should have roughly the same amount of attacks.**

Which means if we were clustering these into three groups and two groups we could check against the predictions grouped them by their actual prediction values and then just do a count to see if there
was three even groups or two even groups.

In [30]:
model_k3.transform( scaled_data).select('prediction').show(5)

+----------+
|prediction|
+----------+
|         2|
|         0|
|         2|
|         2|
|         0|
+----------+
only showing top 5 rows



In [33]:
model_k3.transform( scaled_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         2|   83|
|         0|   84|
+----------+-----+



Based on this, you can see that overall the number is not even. We have clue that they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks.

So, in a three hacker situation, the result should give the same amount of attacks, but what we see  now, as far as the behavior goes, it is not an even split. Let's see what our kmeans2 looks like. 

In [35]:
model_k2.transform( scaled_data).groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+



Yess! We have the actual truth.

We have an exact even split 167 between 1 and 0. 

**Conclusion**: We have two hackers that attack the company!!!!